<!--NAVIGATION-->
<a href="https://colab.research.google.com/github/jwillis0720/sadie/blob/airr_c/notebooks/airr_c/SADIE_DEMO.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

In [4]:
def get_demo_files() -> None:
    """Get the demonstration files for AIRR-C 2022"""
    !pip install git+https://github.com/jwillis0720/sadie.git@26887ee94a91ac60dd35a43a371944b7cab7a8a6
    #print("hello govn")


import sys
if "google.colab" in sys.modules:
    get_demo_files()

In [5]:
get_demo_files()

~Your custom Functions on SiLiHaL
   __z
   __z_add
   __z_clean
   __z_complete
   fish_prompt
   fisher
~~~Live long and prosper~~~
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
  Cloning https://github.com/jwillis0720/sadie.git (to revision 26887ee94a91ac60dd35a43a371944b7cab7a8a6) to /private/var/folders/vj/hbsnq3f94vx3mn2scpw2b86w0000gn/T/pip-req-build-3b547a72
  Running command git clone -q https://github.com/jwillis0720/sadie.git /private/var/folders/vj/hbsnq3f94vx3mn2scpw2b86w0000gn/T/pip-req-build-3b547a72
  Running command git rev-parse -q --verify 'sha^26887ee94a91ac60dd35a43a371944b7cab7a8a6'
  Running command git fetch -q https://github.com/jwillis0720/sadie.git 26887ee94a91ac60dd35a43a371944b7cab7a8a6
  Running command git checkout -q 26887ee94a91ac60dd35a43a371944b7cab7a8a6
  Resolved https://github.com/jwillis0720/sadie.git to commit 26887ee94a91ac60dd35a43a371944b7cab7a8a6
  Created wheel for sadie-antibody: filename=sadie_antibody-0.4.11

In [ ]:
%load_ext lab_black

from sadie.airr import Airr
from pprint import pprint
from Bio.Seq import Seq
from Bio import SeqIO
from sadie.receptor.rearrangment import ReceptorChain
from sadie.airr.airrtable.constants import IGBLAST_AIRR

airr_api = Airr("human")

# Input
vrc01_seq_heavy = SeqIO.read("vrc01_heavy.fasta", "fasta").seq.__str__()
vrc01_seq_light = SeqIO.read("vrc01_light.fasta", "fasta").seq.__str__()
single_result_heavy = airr_api.run_single("VRC01_heavy", vrc01_seq_heavy)
single_result_heavy_slice = single_result_heavy.iloc[0]
single_result_light = airr_api.run_single("VRC01_light", vrc01_seq_light)
single_result_light_slice = single_result_light.iloc[0]


print(single_result_heavy_slice.to_receptor_chain_object())

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
input_sequence
--------------
sequence_id : VRC01_heavy
sequence : CAGGTGCAGCTGGTGCAGTCTGGGGGTCAGATGAAGAAGCCTGGCGAGTCGATGAGAATTTCTTGTCGGGCTTCTGGATATGAATTTATTGATTGTACGCTAAATTGGATTCGTCTGGCCCCCGGAAAAAGGCCTGAGTGGATGGGATGGCTGAAGCCTCGGGGGGGGGCCGTCAACTACGCACGTCCACTTCAGGGCAGAGTGACCATGACTCGAGACGTTTATTCCGACACAGCCTTTTTGGAGCTGCGCTCGTTGACAGTAGACGACACGGCCGTCTACTTTTGTACTAGGGGAAAAAACTGTGATTACAATTGGGACTTCGAACACTGGGGCCGGGGCACCCCGGTCATCGTCTCATCAGGGGGG
raw_sequence : None
sequence_aa : None
category : category='input'


primary_annotations
-------------------
rev_comp : False
productive : True
vj_in_frame : True
stop_codon : False
complete_vdj : True
locus : IGH
v_call : IGHV1-2*02
d_call : IGHD3-16*01,IGHD3-16*02
d2_call : None
j_call : IGHJ1*01
c_call : None
v_call_top : IGHV1-2*02
v_call_top_gene : None
v_call_top_allele : None
d_call_top : IGHD3-16*01
d_call_gene : None
d_call_allele : None
j_call_top : IGHJ1*01
j_ca

In [ ]:
single_result_slice = single_result_light.iloc[0]

In [ ]:
single_result_slice

sequence_id                                                                VRC01_light
sequence                             GAAATTGTGTTGACACAGTCTCCAGGCACCCTGTCTTTGTCTCCAG...
species                                                                          human
locus                                                                              IGK
stop_codon                                                                       False
                                                           ...                        
v_penalty                                                                           -1
d_penalty                                                                           -1
j_penalty                                                                           -2
germline_alignment_aa_corrected                                                   True
v_germline_alignment_aa_corrected                                                 True
Name: 0, Length: 112, dtype: object

# Do it from convienience functions

In [ ]:
print(ReceptorChain.from_single("vrc01_heavy", vrc01_seq_heavy))

input_sequence
--------------
sequence_id : vrc01_heavy
sequence : CAGGTGCAGCTGGTGCAGTCTGGGGGTCAGATGAAGAAGCCTGGCGAGTCGATGAGAATTTCTTGTCGGGCTTCTGGATATGAATTTATTGATTGTACGCTAAATTGGATTCGTCTGGCCCCCGGAAAAAGGCCTGAGTGGATGGGATGGCTGAAGCCTCGGGGGGGGGCCGTCAACTACGCACGTCCACTTCAGGGCAGAGTGACCATGACTCGAGACGTTTATTCCGACACAGCCTTTTTGGAGCTGCGCTCGTTGACAGTAGACGACACGGCCGTCTACTTTTGTACTAGGGGAAAAAACTGTGATTACAATTGGGACTTCGAACACTGGGGCCGGGGCACCCCGGTCATCGTCTCATCAGGGGGG
raw_sequence : None
sequence_aa : None
category : category='input'


primary_annotations
-------------------
rev_comp : False
productive : True
vj_in_frame : True
stop_codon : False
complete_vdj : True
locus : IGH
v_call : IGHV1-2*02
d_call : IGHD3-16*01,IGHD3-16*02
d2_call : None
j_call : IGHJ1*01
c_call : None
v_call_top : IGHV1-2*02
v_call_top_gene : None
v_call_top_allele : None
d_call_top : IGHD3-16*01
d_call_gene : None
d_call_allele : None
j_call_top : IGHJ1*01
j_call_top_gene : None
j_call_top_allele : None
c_call_allele : None
species : human
categ

In [ ]:
print(ReceptorChain.from_single("vrc01_light", vrc01_seq_light))

ValidationError: 6 validation errors for AlignmentPositions
d_sequence_start
  value is not a valid integer (type=type_error.integer)
d_sequence_end
  value is not a valid integer (type=type_error.integer)
d_germline_start
  value is not a valid integer (type=type_error.integer)
d_germline_end
  value is not a valid integer (type=type_error.integer)
d_alignment_start
  value is not a valid integer (type=type_error.integer)
d_alignment_end
  value is not a valid integer (type=type_error.integer)